# This notebook cleans the data extracted from the Second Chamber API

In this file, the data is selected, cleaned and made ready to directly implement in the IBM Cloud functions. 
The final result is a new JSON file.

In [368]:
#importing the libraries
import pandas as pd #to read pandas
import json #to read the JSON file

In [369]:
motions_data = pd.read_json('jsons/moties-30-5.json', orient='columns') #importing the JSON file 
activities_data = pd.read_json('jsons/Activiteit-31-5.json', orient='columns') #importing the JSON file 

In [370]:
motions_all = pd.read_json((motions_data['value']).to_json(), orient='index') #and converting it to a dataframe
activities_all = pd.read_json((activities_data['value']).to_json(), orient='index') #and converting it to a dataframe


In [371]:
#selecting the relavent columns from the original file
#the selection is based on the feedback received from the user tests

motions = motions_all[['Titel','Soort','Onderwerp','Datum']] #selecting the columns
acts = activities_all[['Soort','Onderwerp','Datum','Noot','Besloten','Voortouwnaam']] #selecting the columns

motions.head(5)

,Titel,Soort,Onderwerp,Datum
0,Wijziging van de begrotingsstaten van het Mini...,Motie,Motie van het lid Amhaouch c.s. over steun voo...,2020-05-18T00:00:00+02:00
1,Infectieziektenbestrijding,Motie,Motie van de leden Jetten en Pieter Heerma ove...,2020-05-20T00:00:00+02:00
10,Infectieziektenbestrijding,Motie,Motie van het lid Wilders c.s. over zorgmedewe...,2020-04-16T00:00:00+02:00
11,Infectieziektenbestrijding,Motie,Motie van de leden Van Haga en Hiddema over ge...,2020-05-07T00:00:00+02:00
12,Infectieziektenbestrijding,Motie,Motie van het lid Van Brenk over een parlement...,2020-05-07T00:00:00+02:00


<h2>Note</h2>
<br>
As seen above, some of the data is not readable e.g, date. <br>Also, for the IBM implementation, additional columns are required such as 'applicants' and 'theme'.

In [372]:
#adding the column 'Indiender' (applicants)
motions = motions[motions['Onderwerp'].notnull()].copy() #adding a new column to display the applicant, this will be extracted from 'onderwerp'
motions['Indieners'] = motions['Onderwerp'].astype(str) #converted into strings
motions['Indieners'] = motions['Indieners'].map(lambda x: x.lstrip('Motie van het lid').rstrip('aAbBcC')) #remove text
motions['Indieners'] = motions['Indieners'].str.split('c.s').str[0] #remove text
motions['Indieners'] = motions['Indieners'].str.split('over').str[0] #remove text


motions.head(5)

,Titel,Soort,Onderwerp,Datum,Indieners
0,Wijziging van de begrotingsstaten van het Mini...,Motie,Motie van het lid Amhaouch c.s. over steun voo...,2020-05-18T00:00:00+02:00,Amhaouch
1,Infectieziektenbestrijding,Motie,Motie van de leden Jetten en Pieter Heerma ove...,2020-05-20T00:00:00+02:00,Jetten en Pieter Heerma
10,Infectieziektenbestrijding,Motie,Motie van het lid Wilders c.s. over zorgmedewe...,2020-04-16T00:00:00+02:00,Wilders
11,Infectieziektenbestrijding,Motie,Motie van de leden Van Haga en Hiddema over ge...,2020-05-07T00:00:00+02:00,Van Haga en Hiddema
12,Infectieziektenbestrijding,Motie,Motie van het lid Van Brenk over een parlement...,2020-05-07T00:00:00+02:00,Van Brenk


In [373]:
#transforming the 'date'
import datetime
import locale #to translate the date
locale.setlocale(locale.LC_TIME, "nl_NL") #set to Dutch


motions['Datum'] = motions['Datum'].str.split('T').str[0] #removing the time
motions['Datum'] = pd.to_datetime(motions.Datum) #converting the string to Date
motions = motions.sort_values(by='Datum', ascending=False) #moste recent motions first
#motions['Datum'] = motions['Datum'].dt.strftime('%Y-%b-%d') #transforms the Date back to string
motions['Datum'] = motions['Datum'].dt.strftime('%d %B, %Y') #transforms the Date back to string, and more makes it readable

motions.head()

,Titel,Soort,Onderwerp,Datum,Indieners
3,Noodpakket banen en economie,Motie,Motie van de leden Van Haga en Baudet over de ...,"28 mei, 2020",Van Haga en Baudet
16,Noodpakket banen en economie,Motie,Motie van het lid Pieter Heerma c.s. over een ...,"28 mei, 2020",Pieter Heerma
43,Financieel jaarverslag van het Rijk 2019,Motie,Motie van het lid Kuzu over aandacht voor doel...,"26 mei, 2020",Kuzu
45,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Smeulders en Bromet over de...,"25 mei, 2020",Smeulders en Bromet
15,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Van Kent en Bruins over de ...,"25 mei, 2020",Van Kent en Bruins


<h2>Note</h2>

Some of the data is missing because it could not be extracted from the API, due to lack of knowlegde. However, the columns were added to provide a preview of the research's intention. The selection is based on the feedback received from the user tests.

In [374]:
motions['Status'] = 'tegen/voor gestemd'  #voting behaviour
motions['Inhoud'] = 'meer informatie' #additional informtion 
motions = motions.rename(columns = {'Titel':'Thema'}) #theme's were created to provide an overview

In [375]:
motions

,Thema,Soort,Onderwerp,Datum,Indieners,Status,Inhoud
3,Noodpakket banen en economie,Motie,Motie van de leden Van Haga en Baudet over de ...,"28 mei, 2020",Van Haga en Baudet,tegen/voor gestemd,meer informatie
16,Noodpakket banen en economie,Motie,Motie van het lid Pieter Heerma c.s. over een ...,"28 mei, 2020",Pieter Heerma,tegen/voor gestemd,meer informatie
43,Financieel jaarverslag van het Rijk 2019,Motie,Motie van het lid Kuzu over aandacht voor doel...,"26 mei, 2020",Kuzu,tegen/voor gestemd,meer informatie
45,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Smeulders en Bromet over de...,"25 mei, 2020",Smeulders en Bromet,tegen/voor gestemd,meer informatie
15,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Van Kent en Bruins over de ...,"25 mei, 2020",Van Kent en Bruins,tegen/voor gestemd,meer informatie
30,Infectieziektenbestrijding,Motie,Motie van de leden Baudet en Van Haga over ond...,"20 mei, 2020",Baudet en Van Haga,tegen/voor gestemd,meer informatie
38,Wijziging van de begrotingsstaat van Koninkrij...,Motie,Motie van het lid Diertens over instellen van ...,"20 mei, 2020",Diertens,tegen/voor gestemd,meer informatie
37,Infectieziektenbestrijding,Motie,Motie van het lid Azarkan c.s. over justitiële...,"20 mei, 2020",Azarkan,tegen/voor gestemd,meer informatie
33,Infectieziektenbestrijding,Motie,Motie van het lid Van Brenk over gemeenten com...,"20 mei, 2020",Van Brenk,tegen/voor gestemd,meer informatie
17,Infectieziektenbestrijding,Motie,Motie van het lid Krol over lessen trekken uit...,"20 mei, 2020",Krol,tegen/voor gestemd,meer informatie


In [376]:
#Due to lack of access and knowlegde to the API, additional data was typed over from the Second Chamber website. 
#Covid-19 bills were added
motions.loc[motions.index.max()+1] = ['Vaste commissie voor Economische Zaken en Klimaat',
                                      'Wetsvoorstel',
                                      'Tijdelijke wet informatieverstrekking RIVM in verband met COVID-19',
                                      '29 mei, 2020',
                                      'M.C.G. Keijzer',
                                      'Voorbereiding',
                                      'Dit wetsvoorstel creëert een grondslag in hoofdstuk 14 van de Telecommunicatiewet om aanbieders van openbare mobiele telecommunicatienetwerken te verplichten informatie te verstrekken aan het Rijksinstituut Rijksinstituut voor volksgezondheid en milieu (RIVM) ter ondersteuning bij de bestrijding van de het coronavirus. Het is voor het RIVM noodzakelijk om drukte en verplaatsingen op bevolkingsniveau te kunnen monitoren. Het gaat nadrukkelijk niet om het volgen maar om het tellen van personen.']
motions

,Thema,Soort,Onderwerp,Datum,Indieners,Status,Inhoud
3,Noodpakket banen en economie,Motie,Motie van de leden Van Haga en Baudet over de ...,"28 mei, 2020",Van Haga en Baudet,tegen/voor gestemd,meer informatie
16,Noodpakket banen en economie,Motie,Motie van het lid Pieter Heerma c.s. over een ...,"28 mei, 2020",Pieter Heerma,tegen/voor gestemd,meer informatie
43,Financieel jaarverslag van het Rijk 2019,Motie,Motie van het lid Kuzu over aandacht voor doel...,"26 mei, 2020",Kuzu,tegen/voor gestemd,meer informatie
45,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Smeulders en Bromet over de...,"25 mei, 2020",Smeulders en Bromet,tegen/voor gestemd,meer informatie
15,Initiatiefnota van de leden Segers en Marijnis...,Motie,Motie van de leden Van Kent en Bruins over de ...,"25 mei, 2020",Van Kent en Bruins,tegen/voor gestemd,meer informatie
30,Infectieziektenbestrijding,Motie,Motie van de leden Baudet en Van Haga over ond...,"20 mei, 2020",Baudet en Van Haga,tegen/voor gestemd,meer informatie
38,Wijziging van de begrotingsstaat van Koninkrij...,Motie,Motie van het lid Diertens over instellen van ...,"20 mei, 2020",Diertens,tegen/voor gestemd,meer informatie
37,Infectieziektenbestrijding,Motie,Motie van het lid Azarkan c.s. over justitiële...,"20 mei, 2020",Azarkan,tegen/voor gestemd,meer informatie
33,Infectieziektenbestrijding,Motie,Motie van het lid Van Brenk over gemeenten com...,"20 mei, 2020",Van Brenk,tegen/voor gestemd,meer informatie
17,Infectieziektenbestrijding,Motie,Motie van het lid Krol over lessen trekken uit...,"20 mei, 2020",Krol,tegen/voor gestemd,meer informatie


In [377]:
#converting the datafrome into a new JSON file

motions.to_json(r'jsons/moties-bills-1.json', orient='table') 